In [ ]:
from IPython.core.display import display, HTML
display(HTML('<style>.rendered_html {font-size: 16px;}</style>'))

# Introduction

Conjugated machine learning is a concept where fundamental chemical laws in the form of strict mathematical relationships are directly integrated with machine learning algorithms. Conjugated models correctly predict the properties of molecules so that they satisfy the integrated chemical law.

<img src='img/tau_img/figure_1.png' width='450'/>

In this example, we consider the application of conjugated learning to simultaneous prediction of the tautomeric constant ($logK_{T}$) and the acidity ($pK_{a}$) of the corresponding tautomers. 


## Individual Ridge Regression: $pK_{a}$ prediction
    
In Ridge Regression, $pK_{a}$ prediction is calculated as

$$ y_{A}^{pred} = Xw $$

where $X$ is the matrix of descriptors encoding chemical compounds and $w$ are the regression weights.
After minimising the error function

$$ E_{A}(w) = (y_{A}^{exp} - y_{A}^{pred})^T(y_{A}^{exp} - y_{A}^{pred}) = 
             (y_{A}^{exp} - Xw)^T(y_{A}^{exp} - Xw) => min $$

we can calculate the regression weights using the analytical expression

$$ w = (X^{T}X + \lambda I)^{-1}X^{T}y_{A}^{exp}$$

where $X$ is the descriptor matrix of training compounds and $y_{A}^{exp}$ are the experimental $pK_{a}$ values. $\lambda$ is the regularization coefficient, which controls the complexity of the model.

## Individual Ridge Regression: $logK_{T}$ prediction

The tautomeric equilibrium constant can be calculated as the difference in $pK_{a}$ of the tautomers

$$ logK_{T} = pKa(product) - pKa(reagent) $$

$$ logK_{T} = pKa(2) - pKa(1) $$

In fact, the $logK_{T}$ can be modelled and predicted in two ways:

**1**. Calculation of $logK_{T}$ via predicted $pK_{a}$ of tautomers

The prediction of the tauomeric constant $y_{T}^{pred}$ can be calculated with predicted $pK_{a}$ of tautomers

$$ y_{T}^{pred} = y_{A}^{pred}(2) - y_{A}^{pred}(1) = X_{2}w - X_{1}w = (X_{2} - X_{1})w $$

$$ w = (X^{T}X + \lambda I)^{-1}X^{T}y_{A}^{exp} $$

All we need here is a $pK_{a}$ model.



**2**. The tauomeric constant $y_{T}^{pred}$ can be modelled and predicted directly from the difference descriptor matrix

$$ X_{21} = X_{2} - X_{1}$$

$$ y_{T}^{pred} = X_{21}w $$

$$ w = (X_{21}^{T}X_{21} + \lambda I)^{-1}X_{21}^{T}y_{T}^{exp} $$

where $X_{21}$ is the difference descriptor matrix of training tautomeric reactions and $y_{T}^{exp}$ are the experimental $logK_{T}$ values.

## Conjugated Ridge Regression: $pK_{a}$ and $logK_{T}$ prediction

The two approaches to $pK_{a}$ and $logK_{T}$ prediction can be combined into a conjugated model that is trained simultaneously on both $pK_{a}$ and $logK_{T}$ datasets.


Error function for predicting $pK_{a}$
$$ E_{A}(w) = (y_{A}^{exp} - y_{A}^{pred})^T(y_{A}^{exp} - y_{A}^{pred}) $$

$$ E_{A}(w) = (y_{A}^{exp} - Xw)^T(y_{A}^{exp} - Xw) => min $$

Error function for predicting $logK_{T}$
$$ E_{T}(w) = (y_{T}^{exp} - y_{T}^{pred})^T(y_{T}^{exp} - y_{T}^{pred}) $$

$$ E_{T}(w) = (y_{T}^{exp} - (X_{2} - X_{1})w)^T(y_{T}^{exp} - (X_{2} - X_{1})w) => min $$

Conjugated error function for predicting $pK_{a}$ and $logK_{T}$

$$ E(w) = \alpha E_{T}(w) + (1 - \alpha)E_{A}(w) + \lambda w^{T}w => min $$ 

where $\lambda$ is a regularization coefficient, while $\alpha$ takes values from 0 to 1 and controls the trade-off between minimizing prediction errors of tautomeric constants vs acidity constants. Regression weights in the conjugated model are calculated as:

$$ w = [\alpha(X_{2} - X_{1})^{T}(X_{2} - X_{1}) + (1 - \alpha)X^{T}X + \lambda I]^{-1}
\alpha(X_{2} - X_{1})^Ty_{T}^{exp} + (1 - \alpha)X^{T}y_{A}^{exp}$$

# 0. Code installation

Using **conda** and **pip** is the easiest way to install all required packages. Create a new environment (named "exp") with Python 3.6. <br/>

Run these commands in the command line:

`conda create -n exp python=3.6`<br/>
`conda activate exp` <br/>

Install PyTorch package: <br/>

`conda install pytorch torchvision torchaudio cudatoolkit=11.3 -c pytorch` <br/>
`pip install torch_optimizer` <br/>

Install software to calculate 2D fragment descriptors: <br/>

`pip install CGRTools CIMTools` <br/>

If there is a problem with installing `CGRTools` or `CIMTools` on Windows, also install the library:

`conda install libpython m2w64-toolchain -c msys2`

# 1. Dataset preparation

For the demonstration we have two data sets: experimental $pK_{a}$ of organic compounds and the $logK_{T}$ of binary tautomeric reactions. Both data sets have a fixed structure:

**Compound data set - SDF file**.

MOL_BLOCK - *Compound structure* <br/>
\>  \<temperature\> - *temperature, K* <br/>
293.15 <br/>
\>  \<additive.1\> - *organic solvent (or pure water)* <br/>
water <br/>
\>  \<amount.1\> - *the amount of organic solvent in solution with water [0, 1]* <br/>
1.0 <br/>
\>  \<tabulated_constant\> - *experimental $pK_{a}$* <br/>
11.03

<br/>

**Reaction data set - RDF file**.


\\$MOL - *Reagent structure* <br/>
\\$MOL - *Product strucutre* <br/>
\\$DTYPE temperature - *temperature, K* <br/>
\\$DATUM 353.0 <br/>
\\$DTYPE additive.1 - *organic solvent (or pure water)* <br/>
\\$DATUM ethanol <br/>
\\$DTYPE amount.1 - *the amount of organic solvent in solution with water [0, 1]* <br/>
\\$DATUM 1.0 <br/>
\\$DTYPE tabulated_constant - *experimental $logK_{T}$* <br/>
\\$DATUM -1.54 <br/>

In [ ]:
import os
import pickle
from CGRtools import SDFRead, RDFRead

In [ ]:
mols = SDFRead(os.path.join('data', 'tau_data', 'acidity_data.sdf')).read()
reacts = RDFRead(os.path.join('data', 'tau_data', 'tautomerism_data.rdf')).read()

print(f'The acidity data set contains {len(mols)} molecules')
print(f'The tautomerism data set contains {len(reacts)} reactions')

The acidity of minor tautomers is often unknown, but can be calculated using the equation

$$ logK_{T} = pKa(2) - pKa(1) $$

$$ pKa(2) = logK_{T} + pKa(1) $$

Thus, to calculate $pKa(2)$ it is necessary to have the experimental $logK_{T}$ of the reaction and the $pKa(1)$ of the major tautomer. In addition, the $logK_{T}$ and $pKa(1)$ must be measured under the closest conditions (temperature, solvent, amount of solvent). We extracted two such examples from the $logK_{T}$ and $pKa$ datasets. These are keto-enol tautomerism reactions: one in **1,4-dioxane** and the other in **methanol**:

In [ ]:
reacts[308]

In [ ]:
mols[160]

In [ ]:
with open(os.path.join('data', 'tau_data', 'data_itersection.pickle'), 'rb') as f:
    data_stat = pickle.load(f)
data_stat

<img src='img/tau_img/figure_2.png' width='450'/>

We put these two reactions in the test set, the others in the training set.

In [ ]:
with open(os.path.join('data', 'tau_data', 'data_split.pickle'), 'rb') as f:
    data_split = pickle.load(f)
MOLS_TRAIN, MOLS_TEST = data_split['mols_train'], data_split['mols_test']
REACTS_TRAIN, REACTS_TEST = data_split['reacts_train'], data_split['reacts_test']

REAGENTS_TRAIN, REAGENTS_TEST = [], []
PRODUCTS_TRAIN, PRODUCTS_TEST = [], []
for i in REACTS_TRAIN:
    r, p = i.molecules()
    r.meta.update(i.meta)
    p.meta.update(i.meta)
    REAGENTS_TRAIN.append(r)
    PRODUCTS_TRAIN.append(p)
    
for i in REACTS_TEST:
    r, p= i.molecules()
    r.meta.update(i.meta)
    p.meta.update(i.meta)
    REAGENTS_TEST.append(r)
    PRODUCTS_TEST.append(p)

In [ ]:
def get_df(y1_test, y2_test, y1_pred, y2_pred):

    col_list = [('PRODUCT_PKA','EXP'), ('PRODUCT_PKA','PRED'),
                ('REAGENT_PKA','EXP'), ('REAGENT_PKA','PRED'),
                ('LOGKT','EXP'), ('LOGKT','PRED')]

    col_list = pd.MultiIndex.from_tuples(col_list)
    df = pd.DataFrame(columns=col_list)
    #
    df[('PRODUCT_PKA', 'EXP')] = y2_test
    df[('PRODUCT_PKA', 'PRED')] = y2_pred.flatten()

    df[('REAGENT_PKA', 'EXP')] = y1_test
    df[('REAGENT_PKA', 'PRED')] = y1_pred.flatten()

    df[('LOGKT', 'EXP')] = df[('PRODUCT_PKA', 'EXP')] - df[('REAGENT_PKA', 'EXP')]
    df[('LOGKT', 'PRED')] = df[('PRODUCT_PKA', 'PRED')] - df[('REAGENT_PKA', 'PRED')]
    
    return df.round(2)

# 2 Model building
## 2.1 Individual Ridge Regression: $pK_{a}$ prediction

The analytical expression for calculating regression weights is implemented in **IndividualRidge** estimator. Let's try it out.

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import r2_score

from colearn.utils import ISIDAFragmentor
from colearn.estimators.ridge_regression import IndividualRidge 

Now let's calculate the fragment descriptors for the given compounds ($pK_{a}$ dataset) using the **ISIDAFragmentor**

In [ ]:
FRAG = ISIDAFragmentor()
FRAG.fit(MOLS_TRAIN)

XA_TRAIN = FRAG.transform(MOLS_TRAIN)
XA_TEST = FRAG.transform(MOLS_TEST)

YA_TRAIN = np.array([float(i.meta['tabulated_constant']) for i in MOLS_TRAIN])
YA_TEST = np.array([float(i.meta['tabulated_constant']) for i in MOLS_TEST])

print(f'Compounds are encoded with {XA_TRAIN.shape[-1]} fragment descriptors.')

and train the model

In [ ]:
IND_RIDGE_PKA = IndividualRidge(lmb=0.001) # lmb is a regularization coefficient
IND_RIDGE_PKA.fit(XA_TRAIN, YA_TRAIN)

and predict pKa for the test molecules

In [ ]:
YA_PRED = IND_RIDGE_PKA.predict(XA_TEST)

df = pd.DataFrame()
df['EXPERIMENTAL_PKA'] = YA_TEST.flatten()
df['PREDICTED_PKA'] = YA_PRED.flatten()
df.round(2)

In [ ]:
r2 = r2_score(df['EXPERIMENTAL_PKA'], df['PREDICTED_PKA'])
print(f'The determination coefficient on the test set R2 = {r2:.2f}')

Now let's use the same model to predict $logK_{T}$ via predicted $pK_{a}(1)$ and $pK_{a}(2)$ using the equation

$$ logK_{T}^{pred} = pKa(2)^{pred} - pKa(1)^{pred} $$

In [ ]:
X1_TEST = FRAG.transform(REAGENTS_TEST)
X2_TEST = FRAG.transform(PRODUCTS_TEST)

Y1_PRED = IND_RIDGE_PKA.predict(X1_TEST)
Y2_PRED = IND_RIDGE_PKA.predict(X2_TEST)

Y1_TEST = [float(i.meta['tabulated_constant']) for i in MOLS_TEST]
Y2_TEST = [float(i.meta['tabulated_constant']) + float(j.meta['tabulated_constant']) for i, j in zip(REAGENTS_TEST, MOLS_TEST)]

In [ ]:
IND_RIDGE_PKA_RES = get_df(Y1_TEST, Y2_TEST, Y1_PRED, Y2_PRED)
IND_RIDGE_PKA_RES

We can see that the individual model accurately predicts the $pK_{a}(1)$ of the major tautomer, but fails to predict the $pK_{a}(12)$ of the minor tautomer. This is typical because the acidities of minor tautomers are often unknown and not present in the training sets. Also, because of the high error in predicting the acidity of the minor tautomer, we have a high error in the calculated $logK_{T}$ value.

## 2.2 Individual Ridge Regression: $logK_{T}$ prediction

The analytical expression for calculating regression weights is implemented in **IndividualRidge** estimator. Let's try it out.

In [ ]:
X1_TRAIN = FRAG.transform(REAGENTS_TRAIN)
X2_TRAIN = FRAG.transform(PRODUCTS_TRAIN)

X21_TRAIN = X2_TRAIN - X1_TRAIN
X21_TEST = X2_TEST - X1_TEST

YT_TRAIN = np.array([float(i.meta['tabulated_constant']) for i in REACTS_TRAIN])

In [ ]:
IND_RIDGE_LOGKT = IndividualRidge(lmb=10)
IND_RIDGE_LOGKT.fit(X21_TRAIN, YT_TRAIN)

In [ ]:
Y1_PRED = IND_RIDGE_LOGKT.predict(X1_TEST)
Y2_PRED = IND_RIDGE_LOGKT.predict(X2_TEST)
YT_PRED = IND_RIDGE_LOGKT.predict(X21_TEST)

In [ ]:
IND_RIDGE_LOGKT_RES = get_df(Y1_TEST, Y2_TEST, Y1_PRED, Y2_PRED)
IND_RIDGE_LOGKT_RES

Direct modelling gives more accurate predictions of $logK_{T}$, but the predicted $pK_{a}$ of tautjmers obviously do not have a physical sense. In addition, the subtraction of $X_{2} - X_{1}$ destroys the solvent and temperature descriptors, making the model unable to capture the dependence of $logK_{T}$ on reaction conditions (we see identical $logK_{T}$ predictions for the same reaction carried out under different conditions).

## 2.3 Conjugated Ridge Regression: $pK_{a}$ and $logK_{T}$  prediction

Conjugated error function for predicting $pK_{a}$ and $logK_{T}$

$$ E(w) = \alpha E_{T}(w) + (1 - \alpha)E_{A}(w) + \lambda w^{T}w => min $$ 

where $\lambda$ is a regularization coefficient, while $\alpha$ takes values from 0 to 1 and controls the trade-off between minimizing prediction errors of tautomeric constants vs acidity constants. Regression weights in the conjugated model are calculated as:

$$ w = [\alpha(X_{2} - X_{1})^{T}(X_{2} - X_{1}) + (1 - \alpha)X^{T}X + \lambda I]^{-1}
\alpha(X_{2} - X_{1})^Ty_{T}^{exp} + (1 - \alpha)X^{T}y_{A}^{exp}$$

The analytical expression for calculating regression weights $w$ is implemented in **TautomerismConjugatedRidge** estimator. Let's try it out.

In [ ]:
from colearn.estimators.ridge_regression import TautomerismConjugatedRidge

The conjugated model is trained simultaneously on both $pK_{a}$ and $logK_{T}$ data sets.

In [ ]:
X1_TRAIN = FRAG.transform(REAGENTS_TRAIN)
X2_TRAIN = FRAG.transform(PRODUCTS_TRAIN)
XA_TRAIN = FRAG.transform(MOLS_TRAIN)

X1_TEST = FRAG.transform(REAGENTS_TEST)
X2_TEST = FRAG.transform(PRODUCTS_TEST)
XA_TEST = FRAG.transform(MOLS_TEST)

YA_TRAIN = np.array([float(i.meta['tabulated_constant']) for i in MOLS_TRAIN])
YA_TEST = np.array([float(i.meta['tabulated_constant']) for i in MOLS_TEST])

YT_TRAIN = np.array([float(i.meta['tabulated_constant']) for i in REACTS_TRAIN])
YT_TEST = np.array([float(i.meta['tabulated_constant']) for i in REACTS_TEST])

In [ ]:
CONJ_RIDGE = TautomerismConjugatedRidge(alpha=0.9, lmb=0.001)
CONJ_RIDGE.fit(X1_TRAIN, X2_TRAIN, XA_TRAIN, YT_TRAIN, YA_TRAIN)

In [ ]:
Y1_PRED = CONJ_RIDGE.predict_acidity(X1_TEST)
Y2_PRED = CONJ_RIDGE.predict_acidity(X2_TEST)

In [ ]:
CONJ_RIDGE_RES = get_df(Y1_TEST, Y2_TEST, Y1_PRED, Y2_PRED)
CONJ_RIDGE_RES

We can now clearly see that the conjugated model accurately predicts the $pK_{a}$ not only for the major tautomer, but also for the minor one! Consequently, the prediction accuracy of the $logK_{T}$ is also significantly higher.

But we still see that the model does not capture the dependence of $logK_{T}$ on reaction conditions. This problem can be solved with conjugated learning based on neural networks.

## 2.4 Conjugated Neural Network: $pK_{a}$ and $logK_{T}$  prediction

We developed a special architecture of “twin” neural networks based on fully connected feed-forward multilayer NN with shared values of connection weights $w$.

<img src='img/tau_img/figure_3.png' width='450'/>

In [ ]:
from colearn.estimators.neural_nets.tautomerism_net import TautomerismConjugatedNet

In [ ]:
ndim = (X1_TRAIN.shape[1], 256, 128, 64)

CONJ_NET = TautomerismConjugatedNet(ndim=ndim, alpha=0.9, init_cuda=False)
CONJ_NET.fit(X1_TRAIN, X2_TRAIN, XA_TRAIN, YT_TRAIN, YA_TRAIN, 
             n_epoch=2000, batch_size=9999, lr=0.001, weight_decay=0.001, verbose=False)

In [ ]:
Y1_PRED = CONJ_NET.predict_acidity(X1_TEST)
Y2_PRED = CONJ_NET.predict_acidity(X2_TEST)

In [ ]:
CONJ_NET_RES = get_df(Y1_TEST, Y2_TEST, Y1_PRED, Y2_PRED)
CONJ_NET_RES

We can now see that the conjugated model accurately predicts the $logK_{T}$ for reactions carried out under different conditions. 

**Remark**: In the Ridge Regression we can concatenate the difference descriptor matrix $X_{21}$ and the solvent and temperature descriptors (after subtraction of $X_{2} - X_{1}$), enabling the reaction conditions to be captured in the model training.

Let's look at the final table comparing all approaches and models for the first test reaction

In [ ]:
DF_FINAL = pd.concat([IND_RIDGE_PKA_RES.loc[0:0],
                      IND_RIDGE_LOGKT_RES.loc[0:0],
                      CONJ_RIDGE_RES.loc[0:0],
                      CONJ_NET_RES.loc[0:0]])

DF_FINAL['MODEL'] = ['Individual Ridge pKa', 'Individual Ridge logKT', 'Conjugated Ridge', 'Conjugated Net']
DF_FINAL.set_index('MODEL')

and for the second test reaction

In [ ]:
DF_FINAL = pd.concat([IND_RIDGE_PKA_RES.loc[1:1],
                      IND_RIDGE_LOGKT_RES.loc[1:1],
                      CONJ_RIDGE_RES.loc[1:1],
                      CONJ_NET_RES.loc[1:1]])

DF_FINAL['MODEL'] = ['Individual Ridge pKa', 'Individual Ridge logKT', 'Conjugated Ridge', 'Conjugated Net']
DF_FINAL.set_index('MODEL')